In [105]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
os.chdir('C:\\Users\\junch\\Desktop\\New folder')

# Import data

In [698]:
match_original=pd.read_csv('simulated_data.csv')
match_original.head()

,Unnamed: 0,id,year,Dept,treatment,profit,size,wageindex
0,1,110003.0,2003,M5,0,4926.0,164.0,0.675814
1,2,110003.0,2004,M5,0,5194.0,141.0,0.820107
2,3,110003.0,2005,M5,0,5347.0,152.0,1.194434
3,4,110003.0,2006,M5,0,5502.0,152.0,0.632703
4,5,110003.0,2007,M5,0,5814.0,126.0,0.932007


# Data cleaning

In [699]:
#Missing Value
match_original.isna().sum()

Unnamed: 0     0
id            30
year           0
Dept           0
treatment      0
profit         0
size           0
wageindex      0
dtype: int64

In [700]:
#Two teams' ID are missing and deleted. Missing ID is meaningless for matching.
match_original.dropna(axis=0,inplace=True)
match_original.isna().sum()

Unnamed: 0    0
id            0
year          0
Dept          0
treatment     0
profit        0
size          0
wageindex     0
dtype: int64

In [688]:
#Transfer ID to integer
match_original[['id']] = match_original[['id']].astype(int)

In [689]:
#Split teams to experiment group and control group.
experiment= match_original[match_original['treatment']==1]
control=match_original[match_original['treatment']==0]

In [701]:
#Extract exp_first for matching using experiment group data in the first year
exp_first=pd.merge(experiment,match_original,how='inner', on='id')
cols = [0,2,3,4,5,6,7,8]
exp_first.drop(exp_first.columns[cols],axis=1,inplace=True)
exp_first.columns=['id','year','Dept','treatment','profit','size','wageindex']
exp_first.sort_values(['id','year'],ascending=[1,1],inplace=True)
exp_first = exp_first.groupby(['id']).head(1)
exp_first['treatment']=1
exp_first.head()

,id,year,Dept,treatment,profit,size,wageindex
0,110070,2003,M5,1,4221.0,150.0,0.770716
15,110505,2003,M5,1,5600.0,153.0,0.676197
30,140005,2003,M1,1,14629.0,398.0,1.504247
45,142350,2003,M1,1,18258.0,364.0,1.137640
60,150160,2003,R9,1,8207.0,201.0,1.183509


In [702]:
#Extract con_first for matching using control group data in the first year
control.sort_values(['id','year'],ascending=[1,1],inplace=True)
con_first = control.groupby(['id']).head(1)
#Delete duplicated data
con_first.drop(con_first.columns[0],axis=1,inplace=True)
con=pd.merge(exp_first,con_first,how='right', on='id')
con_first=con[con['treatment_x']!=1]
colss = [1,2,3,4,5,6]
con_first.drop(con_first.columns[colss],axis=1,inplace=True)
con_first.columns=['id','year','Dept','treatment','profit','size','wageindex']
con_first.head()

,id,year,Dept,treatment,profit,size,wageindex
436,110003,2003,M5,0,4926.0,164.0,0.675814
437,110173,2003,M5,0,3428.0,117.0,0.833519
438,110200,2003,M5,0,3361.0,115.0,0.768205
439,110260,2003,M5,0,3753.0,108.0,0.745002
440,110280,2003,M5,0,3302.0,130.0,0.874717


In [703]:
#Reset indexing
exp_first=exp_first.reset_index()
con_first=con_first.reset_index()
exp_first.drop(exp_first.columns[0],axis=1,inplace=True)
con_first.drop(con_first.columns[0],axis=1,inplace=True)

# Greedy Matching Algorithms

In [704]:
#Normalizaiton
from sklearn import preprocessing
exp_first['profit']=preprocessing.scale(exp_first['profit'])
exp_first['size']=preprocessing.scale(exp_first['size'])
exp_first['wageindex']=preprocessing.scale(exp_first['wageindex'])
con_first['profit']=preprocessing.scale(con_first['profit'])
con_first['size']=preprocessing.scale(con_first['size'])
con_first['wageindex']=preprocessing.scale(con_first['wageindex'])

In [705]:
#Greedy Matching

#1. Built two array. The first one is to record ID in experiment group and to match same team in control group.
#2. The second array is for calculating Euclidean distance and extract index of minimum value.
#3. Delete control group which has already been matched becasue one control group only could be matched once.
result1=[]
result2=[]
for i in range(len(exp_first)):
    id_exp=exp_first.iloc[i,0]
    result1.append(id_exp)
    array1=np.array(exp_first.iloc[i,4:])
    dept=exp_first.iloc[i,:]['Dept']
    same_dept=con_first[con_first['Dept']==dept]
    array2_data=same_dept.iloc[:,4:]
    for j in range(len(array2_data)):
        array2=np.array(array2_data.iloc[j,:])
        op=np.linalg.norm(array1-array2)
        distance.append(float(op))        
    index=distance.index(min(distance))  
    id_con=same_dept.iloc[index,0]
    result2.append(id_con)
    if len(exp_first[exp_first['Dept']==dept]) <= len(con_first[con_first['Dept']==dept]):
        con_first = con_first[con_first.id != id_con]
    distance=[]
    


In [706]:
# merge two columns and transfer matching result to csv document 
d={"id_experiment":result1,"id_control":result2}
match=pd.DataFrame(d)
match.to_csv('match.csv', index=False)
match.head(10)

,id_experiment,id_control
0,110070,110200
1,110505,110003
2,140005,141900
3,142350,141095
4,150160,150001
5,160013,160002
6,160500,160350
7,160610,160590
8,160810,160650
9,210170,211185


In [707]:
import collections

print([item for item, count in collections.Counter(result2).items() if count > 1])

[341075, 340145, 340040, 341165, 340195, 340005, 820111, 850235]
